# 1. Importing required packages

In [118]:
from time import strftime, gmtime
import pandas as pd
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys

# 2. Inputing username and number of tweets

In [247]:
username = input('Input a username for searching the tweets: ')

Input a username for searching the tweets: barackobama


In [248]:
limit = int(input('Input the number of tweets to search for (if available): '))

Input the number of tweets to search for (if available): 50


### 3. Parsing the page and writing extracted data to table

In [249]:
pd.options.display.float_format = '{:0,.0f}'.format

browser = webdriver.Chrome()
url = f'https://twitter.com/{username}'
tweets_limit = limit

browser.get(url)
time.sleep(1)

body = browser.find_element_by_tag_name('body')

first_len = 0

while True:
    body.send_keys(Keys.END)
    time.sleep(1)
    tweets = browser.find_elements_by_css_selector('div.tweet')

    second_len = len(tweets)

    if second_len == first_len:
        tweets_limit = second_len
        break
    else:
        first_len = second_len

    if second_len >= tweets_limit:
        break

tweets_table = pd.DataFrame(index=range(1, tweets_limit + 1))
        
if (tweets_limit == 0):
    print('There is no tweets published by this user!')
else:

    # Extracting tweet contents
    for n in range(1, tweets_limit + 1):
        tweets_table.at[n, 'Text'] = tweets[n - 1].find_element_by_css_selector('p.tweet-text').text

        # Checking if tweet contains video or picture
        try:
            tweets[n - 1].find_element_by_css_selector('div.AdaptiveMediaOuterContainer')
            tweets_table.at[n, 'Mediafile'] = 'Yes'
        except NoSuchElementException:
            tweets_table.at[n, 'Mediafile'] = 'No'

        # Checking if tweet contains another tweet quotation
        try:
            tweets[n - 1].find_element_by_css_selector('div.QuoteTweet-container')
            tweets_table.at[n, 'Tweet quotation'] = 'Yes'
        except NoSuchElementException:
            tweets_table.at[n, 'Tweet quotation'] = 'No'

        # Checking number of Likes
        likes = tweets[n - 1].find_element_by_css_selector('button.ProfileTweet-action--unfavorite span.ProfileTweet-actionCount').get_attribute('data-tweet-stat-count')
        if likes == '' or likes == None:
            try:
                tweets[n - 1].find_element_by_css_selector('div.ProfileTweet-action--favorite span.ProfileTweet-actionCount.ProfileTweet-actionCount--isZero')
                tweets_table.at[n, 'Likes'] = int(0)
            except NoSuchElementException:
                tweets_table.at[n, 'Likes'] = int(tweets[n - 1].find_element_by_css_selector('div.ProfileTweet-action--favorite span.ProfileTweet-actionCountForPresentation').text)
        else:
            tweets_table.at[n, 'Likes'] = int(likes)
            
        # Checking number of Retweets
        retweets = tweets[n - 1].find_element_by_css_selector('button.ProfileTweet-actionButton.js-actionRetweet span.ProfileTweet-actionCount').get_attribute('data-tweet-stat-count')
        if retweets == '' or retweets == None:
            try:
                tweets[n - 1].find_element_by_css_selector('div.ProfileTweet-action--retweet span.ProfileTweet-actionCount.ProfileTweet-actionCount--isZero')
                tweets_table.at[n, 'Retweets'] = int(0)
            except NoSuchElementException:
                tweets_table.at[n, 'Retweets'] = int(tweets[n - 1].find_element_by_css_selector('div.ProfileTweet-action--retweet span.ProfileTweet-actionCountForPresentation').text)
        else:
            tweets_table.at[n, 'Retweets'] = int(retweets)
            
        # Checking number of Retweets
        comments = tweets[n - 1].find_element_by_css_selector('span.ProfileTweet-action--reply span.ProfileTweet-actionCount').get_attribute('data-tweet-stat-count')
        if comments == '' or comments == None:
            try:
                tweets[n - 1].find_element_by_css_selector('div.ProfileTweet-action--reply span.ProfileTweet-actionCount.ProfileTweet-actionCount--isZero')
                tweets_table.at[n, 'Comments'] = int(0)
            except NoSuchElementException:
                tweets_table.at[n, 'Comments'] = int(tweets[n - 1].find_element_by_css_selector('div.ProfileTweet-action--reply span.ProfileTweet-actionCountForPresentation').text)
        else:
            tweets_table.at[n, 'Comments'] = int(comments)

    # Date and Time
    stamps = browser.find_elements_by_css_selector('div.tweet span._timestamp')[:tweets_limit]
    tweets_table.insert(column='Date & Time', loc=0, value=[strftime('%d %b %Y %H:%M', gmtime(int(stamp.get_attribute('data-time')))) for stamp in stamps])

    # Author of the tweet
    authors = browser.find_elements_by_css_selector('div.tweet strong.fullname')[:tweets_limit]
    tweets_table.insert(column='Author', loc=1, value=[author.text for author in authors])

    # Twitter account link with '@' 
    usernames = browser.find_elements_by_css_selector('div.tweet a.js-action-profile span.username b')[:tweets_limit]
    tweets_table.insert(column='Twitter user', loc=2, value=[f'@{username.text}' for username in usernames])    

    with pd.option_context('display.max_rows', None, 'display.max_columns', None):
        display(tweets_table)

browser.close()

,Date & Time,Author,Twitter user,Text,Mediafile,Tweet quotation,Likes,Retweets,Comments
1,28 Dec 2018 14:34,Barack Obama,@BarackObama,"As 2018 draws to a close, I’m continuing the t...",No,No,"259,641","44,483","8,778"
2,25 Dec 2018 14:14,Barack Obama,@BarackObama,Enjoy the holiday season with the ones you lov...,Yes,No,"925,892","94,168","24,254"
3,20 Dec 2018 20:55,Barack Obama,@BarackObama,There’s no better time than the holidays to gi...,No,Yes,"202,980","33,603","8,307"
4,19 Dec 2018 21:48,Barack Obama,@BarackObama,Merry Christmas and happy holidays to the extr...,No,Yes,"402,056","67,265","14,320"
5,15 Dec 2018 21:15,Barack Obama,@BarackObama,Today is the last day of open enrollment. That...,Yes,No,"78,936","23,957","5,267"
6,15 Dec 2018 00:58,Barack Obama,@BarackObama,Joe is right. Head to http://healthcare.gov b...,No,Yes,"155,578","33,827","3,728"
7,13 Dec 2018 04:15,Barack Obama,@BarackObama,"As Bobby Kennedy taught us, the thing about ho...",No,Yes,"215,987","44,686","7,028"
8,11 Dec 2018 20:31,The Obama Foundation,@ObamaFoundation,The Cares Family reaches across generational d...,Yes,No,"8,703","2,039",388
9,10 Dec 2018 15:40,Barack Obama,@BarackObama,No jump shots. No ferns. No memes. Not this ti...,Yes,No,"351,063","139,952","11,830"
10,08 Dec 2018 14:00,Get America Covered,@GetUSCovered,Just ONE WEEK LEFT to #GetCovered for 2019. Mo...,Yes,No,"5,031","2,847",393


In [250]:
tweets_table.to_csv('tweets.csv')